# Calculating transportation distribution charges

## 1. Task 1

### 1.1 Task description

Use **pandas** to calculate a transportation distribution charge for four gas meters in the United Kingdom. Save your code in a **Jupyter notebook** and upload to a public repo on **Github** (or any other platform of your choice). While solving this exercise, focus on efficiency - i.e., **use vectorised operations and avoid loops!**
All data needed for the calculation can be found in the Excel file for this exercise.

*Transportation distribution charges are levied by gas distribution companies for the use of their lower pressure pipelines; they cover the cost of physically transporting the gas through the pipeline. This rate is determined depending on a meter's exit zone (gas network region) and its estimated annual quantity (AQ); and it changes over time.*

The daily charge is calculated by finding the correct rate for each meter and day in the forecast and multiplying this rate (in p/kWh) with the day's forecast (in kWh).

Calculate the **total cost per meter** by summing its daily charges for the full forecast period and converting to Pounds (1p = 0.01£).

Calculate the **total consumption per meter** by summing its daily consumption forecast for the full period.
Your result should be a DataFrame of the following form with all numerical values rounded to 2 decimals:

| Meter ID | Total Estimated Consumption (kWh) | Total Cost (£) |
| ----------- | ----------- | ----------- |
| 14676236 | | |
| ... | | |
| ... | | |


<ins>*Example:*</ins>

*Looking at meter **14676236** with exit zone **EA1** and an AQ of **28978 kWh**.*

We can find the correct subset of rates in the rate table by selecting the correct exit zone and annual quantity band according to the meter properties. The AQ band is hereby determined by assuring the AQ is between the minimum AQ (*aq_min_kwh*, included) and the maximum AQ (*aq_max_kwh*, excluded, may be open-ended). For this meter, the following rates are found:

**Rates determined for meter *14676236* :**

| **date** | **exit_zone** | **aq_min_kwh** | **aq_max_kwh** | **rate_p_per_kwh** |
| ----------- | ----------- | ----------- | ----------- | ----------- |
| 2020-04-01 | EA1 | 0 | 73200 | 0.2652 |
| 2020-10-01 | EA1 | 0 | 73200 | 0.2970 |
| 2021-04-01 | EA1 | 0 | 73200 | 0.3327 |
| 2021-10-01 | EA1 | 0 | 73200 | 0.3726 |
| 2022-04-01 | EA1 | 0 | 73200 | 0.4173 |
| 2022-10-01 | EA1 | 0 | 73200 | 0.4674 |
| 2023-04-01 | EA1 | 0 | 73200 | 0.5235 |
| 2023-10-01 | EA1 | 0 | 73200 | 0.5863 |
| 2024-04-01 | EA1 | 0 | 73200 | 0.6566 |


*The rate from 2020-04-01 to 2020-09-30 is 0.2652 p/kWh* \
*The rate from 2020-10-01 to 2021-03-31 is 0.2970 p/kWh* \
*...* \
*The rate from 2024-04-01 onwards is 0.6566 p/kWh*

Calculate the cost per day for each meter by multiplying the forecast for that day (kWh) with the rate for that day (p/kWh) to obtain a cost in p.

**Costs calculated for meter *14676236* :**

On 2020-06-01: \
Cost: 0.2652 p/kWh * 22.070768 kWh = 5.85317 p \
On 2022-02-27 : \
Cost : 0.3726 p/kWh * 39.466673 kWh = 14.70528 p \
etc

### 1.2 Task execution

We start with loading all the available dataframes.

In [1]:
import sys
!{sys.executable} -m pip install pytest

You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.8/bin/python3.8 -m pip install --upgrade pip' command.


In [2]:
import numpy as np
import pandas as pd
import pytest

In [3]:
xl = pd.ExcelFile('gas_test_data.xlsx', engine='openpyxl')
xl.sheet_names

['meter_list', 'forecast_table', 'rate_table']

In [4]:
dict_df_calc = {sheet_name: xl.parse(sheet_name) for sheet_name in xl.sheet_names}

We look at the content of the loaded dataframes.

In [5]:
dict_df_calc['meter_list']

,meter_id,aq_kwh,exit_zone
0,14676236,28978,EA1
1,34509937,78324,SO1
2,50264822,265667,NT1
3,88357331,484399,SE2


In [6]:
dict_df_calc['meter_list'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   meter_id   4 non-null      int64 
 1   aq_kwh     4 non-null      int64 
 2   exit_zone  4 non-null      object
dtypes: int64(2), object(1)
memory usage: 224.0+ bytes


In [7]:
dict_df_calc['forecast_table']

,meter_id,date,kwh
0,14676236,2020-06-01,22.070768
1,14676236,2020-06-02,19.170720
2,14676236,2020-06-03,23.555111
3,14676236,2020-06-04,18.220712
4,14676236,2020-06-05,14.196134
...,...,...,...
3407,88357331,2022-09-27,441.014725
3408,88357331,2022-09-28,441.512055
3409,88357331,2022-09-29,437.240657
3410,88357331,2022-09-30,517.608354


In [8]:
dict_df_calc['forecast_table'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3412 entries, 0 to 3411
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   meter_id  3412 non-null   int64         
 1   date      3412 non-null   datetime64[ns]
 2   kwh       3412 non-null   float64       
dtypes: datetime64[ns](1), float64(1), int64(1)
memory usage: 80.1 KB


In [9]:
dict_df_calc['rate_table']

,date,exit_zone,aq_min_kwh,aq_max_kwh,rate_p_per_kwh
0,2020-04-01,EA1,0,73200.0,0.2652
1,2020-04-01,EA1,73200,732000.0,0.1980
2,2020-04-01,EA1,732000,NaN,0.2875
3,2020-04-01,EA2,0,73200.0,0.2970
4,2020-04-01,EA2,73200,732000.0,0.1524
...,...,...,...,...,...
1135,2024-10-01,WM2,73200,732000.0,0.4537
1136,2024-10-01,WM2,732000,NaN,0.7534
1137,2024-10-01,WM3,0,73200.0,0.7263
1138,2024-10-01,WM3,73200,732000.0,0.6109


In [10]:
dict_df_calc['rate_table'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1140 entries, 0 to 1139
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   date            1140 non-null   datetime64[ns]
 1   exit_zone       1140 non-null   object        
 2   aq_min_kwh      1140 non-null   int64         
 3   aq_max_kwh      760 non-null    float64       
 4   rate_p_per_kwh  1140 non-null   float64       
dtypes: datetime64[ns](1), float64(2), int64(1), object(1)
memory usage: 44.7+ KB


We first want to create *df_rate_table* based on *dict_df_calc['rate_table']* in which the max_date for each of the rates are specified.

In [11]:
unique_dates = dict_df_calc['rate_table']['date'].unique()
unique_dates

array(['2020-04-01T00:00:00.000000000', '2020-10-01T00:00:00.000000000',
       '2021-04-01T00:00:00.000000000', '2021-10-01T00:00:00.000000000',
       '2022-04-01T00:00:00.000000000', '2022-10-01T00:00:00.000000000',
       '2023-04-01T00:00:00.000000000', '2023-10-01T00:00:00.000000000',
       '2024-04-01T00:00:00.000000000', '2024-10-01T00:00:00.000000000'],
      dtype='datetime64[ns]')

In [12]:
# we subtract 1 day of each of the unique dates
max_dates = unique_dates - np.timedelta64(1, 'D')
# the first day is removed to get the max dates in the right position compared to the unique_dates
# and the last max date is put 10 years after the previous date to represent 'onwards'
max_dates = np.append(max_dates[1:len(max_dates)], max_dates[-1] + np.timedelta64(3652, 'D'))
max_dates

array(['2020-09-30T00:00:00.000000000', '2021-03-31T00:00:00.000000000',
       '2021-09-30T00:00:00.000000000', '2022-03-31T00:00:00.000000000',
       '2022-09-30T00:00:00.000000000', '2023-03-31T00:00:00.000000000',
       '2023-09-30T00:00:00.000000000', '2024-03-31T00:00:00.000000000',
       '2024-09-30T00:00:00.000000000', '2034-09-30T00:00:00.000000000'],
      dtype='datetime64[ns]')

In [13]:
# the date and max_date are summarized in df_rate_dates
df_rate_dates = pd.DataFrame()
df_rate_dates['date'] = unique_dates
df_rate_dates['max_date'] = max_dates
df_rate_dates

,date,max_date
0,2020-04-01,2020-09-30
1,2020-10-01,2021-03-31
2,2021-04-01,2021-09-30
3,2021-10-01,2022-03-31
4,2022-04-01,2022-09-30
5,2022-10-01,2023-03-31
6,2023-04-01,2023-09-30
7,2023-10-01,2024-03-31
8,2024-04-01,2024-09-30
9,2024-10-01,2034-09-30


In [14]:
# specify order columns df_rate_table (purely for easier readability)
cols_df_rate_table = np.append(['min_date', 'max_date'], dict_df_calc['rate_table'].columns.to_list()[1:])
cols_df_rate_table

array(['min_date', 'max_date', 'exit_zone', 'aq_min_kwh', 'aq_max_kwh',
       'rate_p_per_kwh'], dtype='<U14')

In [15]:
# creation of df_rate_table based on df_rate_dates and the fact that the date column represents the min date
df_rate_table = (
    dict_df_calc['rate_table']
    .merge(df_rate_dates, on='date')
    .rename(columns={'date': 'min_date'})
    [cols_df_rate_table]
)
# we also have to replace the NaN in aq_max_kwh by a very large number (we put 7320000)
df_rate_table[['aq_max_kwh']] = df_rate_table[['aq_max_kwh']].fillna(7320000).astype(int)
df_rate_table

,min_date,max_date,exit_zone,aq_min_kwh,aq_max_kwh,rate_p_per_kwh
0,2020-04-01,2020-09-30,EA1,0,73200,0.2652
1,2020-04-01,2020-09-30,EA1,73200,732000,0.1980
2,2020-04-01,2020-09-30,EA1,732000,7320000,0.2875
3,2020-04-01,2020-09-30,EA2,0,73200,0.2970
4,2020-04-01,2020-09-30,EA2,73200,732000,0.1524
...,...,...,...,...,...,...
1135,2024-10-01,2034-09-30,WM2,73200,732000,0.4537
1136,2024-10-01,2034-09-30,WM2,732000,7320000,0.7534
1137,2024-10-01,2034-09-30,WM3,0,73200,0.7263
1138,2024-10-01,2034-09-30,WM3,73200,732000,0.6109


We create an extended forecast table *df_forecast_table_ext* to easily execute the calculations.

In [16]:
df_forecast_table_ext = (
    # 1. start with forecast_table
    dict_df_calc['forecast_table']
    # 2. meter_id is the column linking meter_list to forecast_table
    .merge(dict_df_calc['meter_list'], on='meter_id')
    # 3. merge df_rate_table on exit_zone only
    .merge(df_rate_table, on='exit_zone')
    # 4. filter on annual quantity band
    .query('aq_kwh >= aq_min_kwh and aq_kwh < aq_max_kwh')
    # 4b. TEST filter on annual quantity band
    # .groupby(['meter_id', 'aq_kwh', 'aq_min_kwh', 'aq_max_kwh']).count()
    # 5. filter on date
    .query('date >= min_date and date <= max_date')
    # 5b. TEST filter on date
    # .groupby(['date', 'min_date', 'max_date']).count()
    # 6. clean index
    .reset_index(drop=True)
)
df_forecast_table_ext

,meter_id,date,kwh,aq_kwh,exit_zone,min_date,max_date,aq_min_kwh,aq_max_kwh,rate_p_per_kwh
0,14676236,2020-06-01,22.070768,28978,EA1,2020-04-01,2020-09-30,0,73200,0.2652
1,14676236,2020-06-02,19.170720,28978,EA1,2020-04-01,2020-09-30,0,73200,0.2652
2,14676236,2020-06-03,23.555111,28978,EA1,2020-04-01,2020-09-30,0,73200,0.2652
3,14676236,2020-06-04,18.220712,28978,EA1,2020-04-01,2020-09-30,0,73200,0.2652
4,14676236,2020-06-05,14.196134,28978,EA1,2020-04-01,2020-09-30,0,73200,0.2652
...,...,...,...,...,...,...,...,...,...,...
3407,88357331,2022-09-27,441.014725,484399,SE2,2022-04-01,2022-09-30,73200,732000,0.3647
3408,88357331,2022-09-28,441.512055,484399,SE2,2022-04-01,2022-09-30,73200,732000,0.3647
3409,88357331,2022-09-29,437.240657,484399,SE2,2022-04-01,2022-09-30,73200,732000,0.3647
3410,88357331,2022-09-30,517.608354,484399,SE2,2022-04-01,2022-09-30,73200,732000,0.3647


Next, we can create the **df_result** requested in the task description.
1. Calculate the **total cost per meter** by summing its daily charges for the full forecast period and converting to Pounds (1p = 0.01£).
2. Calculate the **total consumption per meter** by summing its daily consumption forecast for the full period.

In [17]:
df_temp = (
    df_forecast_table_ext
    # only retain the relevant columns
    [['meter_id', 'kwh', 'rate_p_per_kwh']]
    # calculate rate in p as indicated by the example
    .assign(rate_p=lambda x: x.kwh * x.rate_p_per_kwh)
    # calculate total cost per meter and total consumption per meter
    .groupby(('meter_id'))
    .sum()
    # from rate in p to rate in pounds
    .assign(rate_pounds=lambda x: x.rate_p * 0.01)
    # drop columns rate_p and rate_p_per_kwh
    .drop(columns=['rate_p_per_kwh', 'rate_p'])
    # rename columns kwh and rate_pounds as requested
    .rename(columns={'kwh': 'Total Estimated Consumption (kWh)', 'rate_pounds': 'Total Cost (£)'})
)
# round all numerical values to two decimals and reset index to get df_result in correct format
df_result = np.round(df_temp, decimals=2).reset_index(drop=False)
df_result

,meter_id,Total Estimated Consumption (kWh),Total Cost (£)
0,14676236,28978.0,100.15
1,34509937,78324.0,275.49
2,50264822,265667.0,731.24
3,88357331,484399.0,1433.16


## 2. Task 2

### 2.1 Task description

Write a function that generates a list of random meters of any size. Examples of valid exit zones can be found in the rate table. You may randomly generate the annual quantity.

### 2.2 Task execution

As a prep for this task, we look at the dict_df_calc['meter_list'] dataframe again and extract the valid exit zones from the *exit_zone* column in *df_rate_table*.

In [18]:
dict_df_calc['meter_list']

,meter_id,aq_kwh,exit_zone
0,14676236,28978,EA1
1,34509937,78324,SO1
2,50264822,265667,NT1
3,88357331,484399,SE2


In [19]:
lst_exit_zones = df_rate_table['exit_zone'].unique()
lst_exit_zones

array(['EA1', 'EA2', 'EA3', 'EA4', 'EM1', 'EM2', 'EM3', 'EM4', 'LC', 'LO',
       'LS', 'LT', 'LW', 'NE1', 'NE2', 'NE3', 'NO1', 'NO2', 'NT1', 'NT2',
       'NT3', 'NW1', 'NW2', 'SC1', 'SC2', 'SC4', 'SE1', 'SE2', 'SO1',
       'SO2', 'SW1', 'SW2', 'SW3', 'WA1', 'WA2', 'WM1', 'WM2', 'WM3'],
      dtype=object)

We can now create the function **create_df_meter_list** based on the list of available exit zones *lst_exit_zones* and the following observations:
- a meter_id is an 8-digit number
- while the lower limit of annual quantity is 0 kwh, we will provide the possibility to overwrite the max annual quantity aq_max_kwh (we take 1000 000 kwh by default) taken into account

In [20]:
from typing import Union

def create_df_meter_list(size: int, aq_max_kwh: float = 1000000, seed: Union[int, None] = None) -> pd.DataFrame:
    """
    Generates a list of random meters of any size, taking only valid exit zones into account.
    The annual quantity is randomly generated as a number between 0 kwh and aq_max_kwh kwh.
    There is a possibility to fix the seed to enable reproducibility.
    """
    np.random.seed(seed)
    df_meter_list = pd.DataFrame(columns=['meter_id', 'aq_kwh', 'exit_zone'])
    # meter_id is an 8-digit number
    df_meter_list['meter_id'] = np.random.randint(low=10000000, high=99999999, size=size)
    # randomly generated aq_kwh (rounded to two decimals after comma)
    df_meter_list['aq_kwh'] = np.round(np.random.uniform(low=0, high=aq_max_kwh, size=size), 2)
    df_meter_list['exit_zone'] = np.random.choice(lst_exit_zones, size=size)

    return df_meter_list

We quickly test the newly created function.

In [21]:
create_df_meter_list(10, seed=42)

,meter_id,aq_kwh,exit_zone
0,75682867,650888.47,SE1
1,66755036,56411.58,SE2
2,66882282,721998.77,NE3
3,31081788,938552.71,NE2
4,23315092,778.77,EA3
5,45788921,992211.56,WM2
6,36735830,617481.51,EM3
7,41632483,611653.16,NT3
8,86737383,7066.31,LC
9,98358551,23062.43,NO2


In [22]:
create_df_meter_list(10, aq_max_kwh = 10000000, seed=42)

,meter_id,aq_kwh,exit_zone
0,75682867,6508884.73,SE1
1,66755036,564115.79,SE2
2,66882282,7219987.72,NE3
3,31081788,9385527.09,NE2
4,23315092,7787.66,EA3
5,45788921,9922115.59,WM2
6,36735830,6174815.10,EM3
7,41632483,6116531.60,NT3
8,86737383,70663.05,LC
9,98358551,230624.25,NO2


## 3. Task 3

### 3.1 Task description

Write a function that generates mock consumption data given a list of meters and a start date and duration (number of days in the forecast). The data may be completely random and it doesn't have to match with the meters' annual quantities either.

### 3.2 Task execution

As a prep, we have a look at *dict_df_calc['forecast_table']* dataframe again and we explore this dataframe a bit further.

In [23]:
dict_df_calc['forecast_table']

,meter_id,date,kwh
0,14676236,2020-06-01,22.070768
1,14676236,2020-06-02,19.170720
2,14676236,2020-06-03,23.555111
3,14676236,2020-06-04,18.220712
4,14676236,2020-06-05,14.196134
...,...,...,...
3407,88357331,2022-09-27,441.014725
3408,88357331,2022-09-28,441.512055
3409,88357331,2022-09-29,437.240657
3410,88357331,2022-09-30,517.608354


In [24]:
# check if every meter_id has values for the same min date and max date
(
    dict_df_calc['forecast_table']
    .loc[(dict_df_calc['forecast_table']['date'] == dict_df_calc['forecast_table'].date.min()) |
         (dict_df_calc['forecast_table']['date'] == dict_df_calc['forecast_table'].date.max())]
    .groupby(['meter_id', 'date'])
    .sum()
)

kwh
meter_id date                  
14676236 2020-06-01   22.070768
         2022-10-01   37.122479
34509937 2020-06-01   78.611504
         2022-10-01   86.866895
50264822 2020-06-01  278.759084
         2022-10-01  328.192979
88357331 2020-06-01  343.883152
         2022-10-01  460.535505

In [25]:
# check for how many days every meter_id has a value
(
    dict_df_calc['forecast_table']
    .groupby(['meter_id'])['date']
    .count()
)

meter_id
14676236    853
34509937    853
50264822    853
88357331    853
Name: date, dtype: int64

In [26]:
# the number of days between the max date and the min date (+ 1 to get the min date included)
dict_df_calc['forecast_table'].date.max() - dict_df_calc['forecast_table'].date.min()

Timedelta('852 days 00:00:00')

In [27]:
# minimum kwh value for a single day in the forecast table
dict_df_calc['forecast_table']['kwh'].min()

2.91877038164736

In [28]:
# maximum kwh value for a single day in the forecast table
dict_df_calc['forecast_table']['kwh'].max()

1169.400411138627

We can now create the function **create_df_forecast_table** based on an available *df_meter_list* (which might be created by **create_df_meter_list**) and the following observations:
- for every meter_id, there is consumption data for every day in the same time interval
- the time interval of *dict_df_calc['forecast_table']* is perfectly determined by the start date (in this case: 2020-06-01) and the duration (in this case: 853 days)
- it seems that the data is ordered as follows:
    1. meter_id (with the order determined by the available df_meter_list)
    2. date (ascending dates starting from the start date)
- while the lower limit of the daily (mock) consumption is 0 kwh, we will provide the possibility to overwrite the max daily (mock) consumption (we take 1200 kwh by default based on the observation that the max kwh value in dict_df_calc['forecast_table'] is equal to 1169.4 kwh)

In [29]:
def create_df_forecast_table(
    df_meter_list: pd.DataFrame,
    start_date: np.datetime64,
    duration: int,
    max_kwh: float = 1200,
    seed: Union[int, None] = None
) -> pd.DataFrame:
    """
    Generates a list of mock consumption data given a list of meters (df_meter_list) and a
    start date (start_date) and duration (number of days in the forecast).
    The daily quantity is randomly generated as a number between 0 kwh and max_kwh kwh.
    There is a possibility to fix the seed to enable reproducibility.
    """
    if duration < 1:
        raise ValueError(f'The duration {duration} (number of days in the forecast) should be a positive number.')

    np.random.seed(seed)
    # we first create df_dates with an additional key column to enable cross join later on
    df_dates = pd.DataFrame(columns=['date', 'key'])
    df_dates['date'] = pd.date_range(start_date, periods=duration, freq="D")
    df_dates['key'] = 0

    # create df_forecast_table starting with df_meter_list[['meter_id']]
    df_forecast_table = (
        df_meter_list[['meter_id']]
        .assign(key=0)
        # cross join df_dates on key
        .merge(df_dates, on='key', how='outer')
        # key is no longer necessary
        .drop(columns=('key'))
        # create kwh column
        .assign(
            kwh = np.random.uniform(
                low=0,
                high=max_kwh,
                size=df_meter_list.shape[0]*df_dates.shape[0]
            )
        )
    )

    return df_forecast_table

We quickly test the newly created function.

In [30]:
# this function should throw a ValueError
with pytest.raises(ValueError):
    create_df_forecast_table(
        df_meter_list=dict_df_calc['meter_list'],
        start_date=np.datetime64('2020-06-01'),
        duration=0,
    )

In [31]:
create_df_forecast_table(
    df_meter_list=dict_df_calc['meter_list'],
    start_date=np.datetime64('2020-06-01'),
    duration=3,
    seed=42
)

,meter_id,date,kwh
0,14676236,2020-06-01,449.448143
1,14676236,2020-06-02,1140.857168
2,14676236,2020-06-03,878.392730
3,34509937,2020-06-01,718.390181
4,34509937,2020-06-02,187.222369
5,34509937,2020-06-03,187.193424
6,50264822,2020-06-01,69.700335
7,50264822,2020-06-02,1039.411375
8,50264822,2020-06-03,721.338014
9,88357331,2020-06-01,849.687093


In [32]:
create_df_forecast_table(
    df_meter_list=dict_df_calc['meter_list'],
    start_date=np.datetime64('2024-10-01'),
    duration=4,
    max_kwh=120,
    seed=42
)

,meter_id,date,kwh
0,14676236,2024-10-01,44.944814
1,14676236,2024-10-02,114.085717
2,14676236,2024-10-03,87.839273
3,14676236,2024-10-04,71.839018
4,34509937,2024-10-01,18.722237
5,34509937,2024-10-02,18.719342
6,34509937,2024-10-03,6.970033
7,34509937,2024-10-04,103.941137
8,50264822,2024-10-01,72.133801
9,50264822,2024-10-02,84.968709


## 4. Task 4

### 4.1 Task description

Write a function that takes as an input a meter list and a consumption forecast table and that calculates the transportation cost table (i.e., best take your logic from task 1 and wrap it in a function). Benchmark this function using meter lists of different sizes and consumption forecasts for periods of different lengths. How does the function scale for larger sets of data?

### 4.2 Task execution

The transportation cost table should be a DataFrame of the following form with all numerical values rounded to 2 decimals:

| Meter ID | Total Estimated Consumption (kWh) | Total Cost (£) |
| ----------- | ----------- | ----------- |
| 14676236 | | |
| ... | | |
| ... | | |

We will use the previously created *df_rate_table* (which was created based on *dict_df_calc['rate_table']* ), from which the top 5 rows and bottom 5 rows are shown for convenience.

In [33]:
df_rate_table.head()

,min_date,max_date,exit_zone,aq_min_kwh,aq_max_kwh,rate_p_per_kwh
0,2020-04-01,2020-09-30,EA1,0,73200,0.2652
1,2020-04-01,2020-09-30,EA1,73200,732000,0.1980
2,2020-04-01,2020-09-30,EA1,732000,7320000,0.2875
3,2020-04-01,2020-09-30,EA2,0,73200,0.2970
4,2020-04-01,2020-09-30,EA2,73200,732000,0.1524


In [34]:
df_rate_table.tail()

,min_date,max_date,exit_zone,aq_min_kwh,aq_max_kwh,rate_p_per_kwh
1135,2024-10-01,2034-09-30,WM2,73200,732000,0.4537
1136,2024-10-01,2034-09-30,WM2,732000,7320000,0.7534
1137,2024-10-01,2034-09-30,WM3,0,73200,0.7263
1138,2024-10-01,2034-09-30,WM3,73200,732000,0.6109
1139,2024-10-01,2034-09-30,WM3,732000,7320000,0.4928


In [35]:
def calculate_transportation_cost_table(
    df_meter_list: pd.DataFrame,
    df_forecast_table: pd.DataFrame,
) -> pd.DataFrame:
    """
    Calculates the transportation cost table given a list of meters (df_meter_list) and a
    consumption forecast table (df_forecast_table).
    """
    df_transportation_cost = (
        # 1. start with forecast_table
        df_forecast_table
        # 2. meter_id is the column linking df_meter_list to df_forecast_table
        .merge(df_meter_list, on='meter_id')
        # 3. merge df_rate_table on exit_zone only
        .merge(df_rate_table, on='exit_zone')
        # 4. filter on annual quantity band
        .query('aq_kwh >= aq_min_kwh and aq_kwh < aq_max_kwh')
        # 5. filter on date
        .query('date >= min_date and date <= max_date')
        # 6. only retain the relevant columns
        [['meter_id', 'kwh', 'rate_p_per_kwh']]
        # 7. calculate rate in p as indicated by the example
        .assign(rate_p=lambda x: x.kwh * x.rate_p_per_kwh)
        # 8. calculate total cost per meter and total consumption per meter
        .groupby(('meter_id'))
        .sum()
        # 9. from rate in p to rate in pounds
        .assign(rate_pounds=lambda x: x.rate_p * 0.01)
        # 10. drop columns rate_p and rate_p_per_kwh
        .drop(columns=['rate_p_per_kwh', 'rate_p'])
        # 11. rename columns kwh and rate_pounds as requested
        .rename(columns={'kwh': 'Total Estimated Consumption (kWh)', 'rate_pounds': 'Total Cost (£)'})
    )
    # return df_transportation_cost with all numerical values rounded to two decimals
    # and reset index afterwards to get df_transportation_cost in the right shape
    return np.round(df_transportation_cost, decimals=2).reset_index(drop=False)

We test the outcome first based on the given *dict_df_calc['meter_list']* and *dict_df_calc['forecast_table']*

In [36]:
df_transportation_cost_test = calculate_transportation_cost_table(
    df_meter_list=dict_df_calc['meter_list'],
    df_forecast_table=dict_df_calc['forecast_table'],
)
df_transportation_cost_test

,meter_id,Total Estimated Consumption (kWh),Total Cost (£)
0,14676236,28978.0,100.15
1,34509937,78324.0,275.49
2,50264822,265667.0,731.24
3,88357331,484399.0,1433.16


In [37]:
df_meter_list = create_df_meter_list(
    size=10,
    seed=42,
)
df_forecast_table = create_df_forecast_table(
    df_meter_list=df_meter_list,
    start_date=np.datetime64('2020-06-01'),
    duration=853,
    max_kwh=120,
    seed=42,
)
df_transportation_cost_test2 = calculate_transportation_cost_table(
    df_meter_list=df_meter_list,
    df_forecast_table=df_forecast_table,
)
df_transportation_cost_test2

,meter_id,Total Estimated Consumption (kWh),Total Cost (£)
0,23315092,50325.49,165.89
1,31081788,50665.11,167.24
2,36735830,51439.99,164.51
3,41632483,50504.16,152.35
4,45788921,50502.38,177.21
5,66755036,51360.63,144.02
6,66882282,51170.09,194.15
7,75682867,50843.70,154.02
8,86737383,50023.65,147.69
9,98358551,49167.25,178.80


In [38]:
df_transportation_cost_test2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 3 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   meter_id                           10 non-null     int64  
 1   Total Estimated Consumption (kWh)  10 non-null     float64
 2   Total Cost (£)                     10 non-null     float64
dtypes: float64(2), int64(1)
memory usage: 368.0 bytes


Next step is to benchmark this function using meter lists of different sizes and consumption forecasts for periods of different lengths. How does the function scale for larger sets of data?

We create a function **benchmark_calc_transportation_cost_table_functions** that accepts the same arguments as **create_df_meter_list** and ****

In [39]:
from typing import Callable

import time


def benchmark_calc_transportation_cost_table_functions(
    func: Callable[[pd.DataFrame, pd.DataFrame], pd.DataFrame],
    size: int,
    duration: int,
    start_date: np.datetime64 = np.datetime64('2024-10-01'),
    max_kwh: float = 3000,
    seed: Union[int, None] = None
) -> None:
    """
    This function benchmarks the function func used to calculate the transportation cost table and
    accepts the arguments used to create df_meter_list and df_forecast_table which are then used as input to func:
    - size of the meter list
    - duration (number of days) of the forecast
    - start date of the forecast (1st of June 2020 by default)
    - the maximum daily quantity in kwh (1200 kwh by default)
    - the seed for reproducibility
    """

    # determine the maximum annual quantity based on the maximum daily quantity
    aq_max_kwh = 366 * max_kwh
    # create meter list
    df_meter_list = create_df_meter_list(
        size=size,
        aq_max_kwh=aq_max_kwh,
        seed=seed,
    )
    # print(df_meter_list)
    # create forecast table
    df_forecast_table = create_df_forecast_table(
        df_meter_list=df_meter_list,
        start_date=start_date,
        duration=duration,
        max_kwh=max_kwh,
        seed=seed,
    )
    # print(df_forecast_table)
    # start timing function
    start_time = time.time()
    # execute function
    df_transportation_cost = func(
        df_meter_list=df_meter_list,
        df_forecast_table=df_forecast_table,
    )
    # print(df_transportation_cost)
    end_time = time.time()

    execution_time = end_time - start_time
    msg = (
        f"Execution time for meter list of size {size} "
        f"and consumption forecasts over period of {duration} days: "
        f"{execution_time} seconds"
    )

    print(msg)

In [40]:
size_lst = [10, 100, 1000]
duration_lst = [10, 100, 1000]

for size in size_lst:
    for duration in duration_lst:
        benchmark_calc_transportation_cost_table_functions(
            func=calculate_transportation_cost_table,
            size=size,
            duration=duration,
            seed=42,
        )

Execution time for meter list of size 10 and consumption forecasts over period of 10 days: 0.01677989959716797 seconds
Execution time for meter list of size 10 and consumption forecasts over period of 100 days: 0.020072221755981445 seconds
Execution time for meter list of size 10 and consumption forecasts over period of 1000 days: 0.07500004768371582 seconds
Execution time for meter list of size 100 and consumption forecasts over period of 10 days: 0.01948404312133789 seconds
Execution time for meter list of size 100 and consumption forecasts over period of 100 days: 0.06807708740234375 seconds
Execution time for meter list of size 100 and consumption forecasts over period of 1000 days: 0.6009202003479004 seconds
Execution time for meter list of size 1000 and consumption forecasts over period of 10 days: 0.07306909561157227 seconds
Execution time for meter list of size 1000 and consumption forecasts over period of 100 days: 0.3938732147216797 seconds
Execution time for meter list of si

We observe the following:
- increasing the consumption forecast period has a significantly larger impact on the execution time when the meter size is larger
- likewise, the meter size has a significantly larger impact on the execution time when the consumption forecast period is larger

## 5. Task 5

### 5.1 Task description

What are your observations after benchmarking? Are there any steps in the cost calculation that can be improved? How would you go about improving the performance of this calculation?

### 5.2 Task execution

I would start with trying to replace the pandas operations by numpy operations wherever possible (which would mainly improve the calculation speed). A first attempt at improving the performance is shown below.

In [41]:
def improved_calculate_transportation_cost_table(
    df_meter_list: pd.DataFrame,
    df_forecast_table: pd.DataFrame,
) -> pd.DataFrame:
    """
    Calculates the transportation cost table given a list of meters (df_meter_list) and a
    consumption forecast table (df_forecast_table).
    """
    arr_temp = (
        # 1. start with forecast_table
        df_forecast_table
        # 2. meter_id is the column linking df_meter_list to df_forecast_table
        .merge(df_meter_list, on='meter_id')
        # 3. merge df_rate_table on exit_zone only
        .merge(df_rate_table, on='exit_zone')
        # 4. filter on annual quantity band
        .query('aq_kwh >= aq_min_kwh and aq_kwh < aq_max_kwh')
        # 5. filter on date
        .query('date >= min_date and date <= max_date')
        # 6. only retain the relevant columns
        [['meter_id', 'kwh', 'rate_p_per_kwh']]
        # 7. transform to 2D numpy array
        .to_numpy()
    )

    # 8. define the arrays for each relevant column
    arr_meter_id = arr_temp[:, 0]
    arr_kwh = arr_temp[:, 1]
    arr_rate_p_per_kwh = arr_temp[:, 2]

    # 9. calculate rate in p as indicated by the example
    arr_rate_p = np.multiply(arr_kwh, arr_rate_p_per_kwh)

    # 10. get unique meter_id's and their indices
    unique_meter_id, meter_id_indices = np.unique(arr_meter_id, return_inverse=True)

    # 11. use np.bincount to sum kwh values based on meter id indices and round to 2 decimals after comma
    sums_kwh = np.round(np.bincount(meter_id_indices, weights=arr_kwh), decimals=2)

    # 12. use np.bincount to sum rate_p values based on meter id indices
    sums_rate_p = np.bincount(meter_id_indices, weights=arr_rate_p)

    # 13. calculate sums_rate_pounds based on sums_rate_p and round to 2 decimals after comma
    sums_rate_pounds = np.round(0.01 * sums_rate_p, decimals=2)

    # 14. create the requested dataframe based on the different arrays
    data = np.column_stack((unique_meter_id, sums_kwh, sums_rate_pounds))
    df_transportation_cost = pd.DataFrame(
        data,
        columns=['meter_id', 'Total Estimated Consumption (kWh)', 'Total Cost (£)']
    )
    # 15. convert meter_id column from float to int type
    df_transportation_cost['meter_id'] = df_transportation_cost['meter_id'].astype(int)

    return df_transportation_cost

We quickly verify whether this function generates the expected result.

In [42]:
df_transportation_cost_test = improved_calculate_transportation_cost_table(
    df_meter_list=dict_df_calc['meter_list'],
    df_forecast_table=dict_df_calc['forecast_table'],
)
df_transportation_cost_test

,meter_id,Total Estimated Consumption (kWh),Total Cost (£)
0,14676236,28978.0,100.15
1,34509937,78324.0,275.49
2,50264822,265667.0,731.24
3,88357331,484399.0,1433.16


In [43]:
size_lst = [10, 100, 1000, 10000, 100000]
duration_lst = [10, 100]

for size in size_lst:
    for duration in duration_lst:
        print("Not optimized: ")
        benchmark_calc_transportation_cost_table_functions(
            func=calculate_transportation_cost_table,
            size=size,
            duration=duration,
            seed=42,
        )
        print("Optimized: ")
        benchmark_calc_transportation_cost_table_functions(
            func=improved_calculate_transportation_cost_table,
            size=size,
            duration=duration,
            seed=42,
        )
        print("\n")

Not optimized: 
Execution time for meter list of size 10 and consumption forecasts over period of 10 days: 0.01679515838623047 seconds
Optimized: 
Execution time for meter list of size 10 and consumption forecasts over period of 10 days: 0.01018071174621582 seconds


Not optimized: 
Execution time for meter list of size 10 and consumption forecasts over period of 100 days: 0.020786046981811523 seconds
Optimized: 
Execution time for meter list of size 10 and consumption forecasts over period of 100 days: 0.016891956329345703 seconds


Not optimized: 
Execution time for meter list of size 100 and consumption forecasts over period of 10 days: 0.0202481746673584 seconds
Optimized: 
Execution time for meter list of size 100 and consumption forecasts over period of 10 days: 0.018362045288085938 seconds


Not optimized: 
Execution time for meter list of size 100 and consumption forecasts over period of 100 days: 0.07036399841308594 seconds
Optimized: 
Execution time for meter list of size 100

For larger (size, period)-combinations, we observe significant improvements in terms of performance. We can definitely find more ways to improve the performance, but this requires further investigation.